In [1]:
import sys
import os
import glob
import json
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np

In [2]:
def find_files(path):
    return glob.glob(path)

In [3]:
def read_json(filename):
    with open(filename) as json_file:
        data = json.load(json_file)
    return data

### Adat beolvasása JSON fájlból

In [4]:
json_data = []

for file in find_files('Data/*'):
    print("Reading file:", file)
    json_data.append(read_json(file))

Reading file: Data/gyakori_szamitastechnika
Reading file: Data/gyakori_egeszseg_20000
Reading file: Data/gyakori_szorakozas_30000
Reading file: Data/gyakori_egeszseg
Reading file: Data/gyakori_allatok_14000


### Dataframe-mé alakítás

In [5]:
data_frames = []

for data in json_data:
    data_frames.append(pd.DataFrame(data))

### Bepillantás a kérdésekbe

In [6]:
for frame in data_frames:
    print(frame.head(2))
    for i in range(0, 25):
        print("--", end="")
    print()

                                              valasz  \
0  Várak régen is voltak. Ha mindhárom tornyot le...   
1  Ahogy írták, az stdio az a C-s függvénykönyvtá...   

                        kategoriak  \
0     [Számítástechnika, Internet]   
1  [Számítástechnika, Programozás]   

                                       hosszu_kerdes  \
0  Miért lett ilyen sz@r a honfoglaló? Régen tök ...   
1  C++-ban melyiket érdemesebb használni? Stdio v...   

                                        rovid_kerdes  \
0                Miért lett ilyen sz@r a honfoglaló?   
1  C++-ban melyiket érdemesebb használni? Stdio v...   

                                          keywords  
0                                [Honfoglaló, vár]  
1  [C++, iostream, konzol, Windows, Visual Studio]  
--------------------------------------------------
                                            keywords  \
0              [inzulinrezisztencia, diéta, reggeli]   
1  [nőgyógyászat, kivizsgálás, fájdalom, magánren...   


### Kérdések megszámolása, átlagos hossz számolása

In [25]:
def calculate_avg(data_frames, label="rovid_kerdes"):
    avg = 0
    count = 0
    
    for frame in data_frames:
        for index, row in frame.iterrows():
            avg += len(row[label])
            count += 1

    avg = avg / count
    
    return avg, count

### Extrém rövid/hosszú kérdések eldobása

In [44]:
def drop_extreme(data_frames, min_, max_, label="rovid_kerdes"):
    for idx, frame in enumerate(data_frames):
        frame = frame[frame[label].map(len) >= min_]
        frame = frame[frame[label].map(len) <= max_]

        data_frames[idx] = frame

In [46]:
long_questions_avg_len, questions_count = calculate_avg(data_frames, "hosszu_kerdes")

print("Number of questions: %d" % questions_count)
print("Average length: %f" % long_questions_avg_len)

Number of questions: 137265
Average length: 326.753950


In [47]:
drop_extreme(data_frames, long_questions_avg_len / 4, long_questions_avg_len * 3, "hosszu_kerdes")
long_questions_avg_len, questions_count = calculate_avg(data_frames, "hosszu_kerdes")

print("Number of questions: %d" % questions_count)
print("Average length: %f" % long_questions_avg_len)

Number of questions: 136245
Average length: 321.648494


In [60]:
target_dir = {}
target_names = []

target_values = []
long_questions = []

for frame in data_frames:
    for idx in frame.index:
        category = frame["kategoriak"][idx][0]
        if category not in target_names:
            target_dir[category] = len(target_dir)
            target_names.append(category)
        target_values.append(target_dir[category])
        long_questions.append(frame["hosszu_kerdes"][idx])
print(target_names)        

['Számítástechnika', 'Egészség', 'Szórakozás', 'Állatok']


In [61]:
import random

paired = list(zip(target_values, long_questions))

random.shuffle(paired)

target_values, long_questions = zip(*paired)

for i in range(0, 5):
    print(target_names[target_values[i]])
    print(long_questions[i])
    print("--------------------------------------")

Állatok
Dzsungáriai törpehörcsög vagy Roborovszki hörcsög? -tartása- Nem tudom eldönteni, mivel mind a kettő cuki KÜLSŐRE :D igazából azt szeretném hogy lehessen simogatni, "babusgatni" melyik jobb választás a két höri közül? És le tudnátok írni a tartását? Csak úgy akarom megvenni,hogy tudom, alkalmas vagyok-e arra hogy tartsak ilyen állatkát :) nem szeretném, hogy rosszul érezze magát. Válaszokat előre is köszönöm.
--------------------------------------
Egészség
Miért van ez? Öt napja kèsik..de a hasamban írgalmas fäjdalom van ....ami szokott elötte lenni..de most nincs semmi mäs ,csak hasfäjäs...Terhesn nem lehetek...nem elek nemi èletet...Szerintetek?
--------------------------------------
Szórakozás
Leiner Laurának a legújabb könyvében (hullócsillag) van egy selfie alkalmazás. Az létezik? Márk és Beki csinálják a képet, és az utolsó képen Márk egy puszit ad Beki arcára. Köszönöm a válaszokat!
--------------------------------------
Egészség
Dohányzás bio módra?  [link] Szeretnék e 